In [2]:
import logging
import pandas as pd
import os
from itertools import chain
import requests
import json
from datetime import datetime
from redcap import Project
import jira
import parse

logging.basicConfig(
    format="%(asctime)s - %(message)s", datefmt="%Y-%m-%d %H:%M:%S", level=logging.INFO
)

# #############################################################################
# redcap api setup
# #############################################################################
api_url = 'https://redcap.chop.edu/api/'
api_key = 'C9F11788CCEAEF6D5AF3EF303C4EC9E4'
project = Project(api_url, api_key, verify_ssl=False)

now = datetime.now()
currentdate = now.strftime("%Y-%m-%d")

# #############################################################################
# Get today's requests from REDCap
# #############################################################################

loaded_reqs = project.export_records(format='df')
loaded_reqs.columns = map(str.upper, loaded_reqs.columns)
loaded_reqs_df = pd.DataFrame(loaded_reqs,columns = ['cc_data_request_id','REQUEST_TITLE', 'DATE_OF_REQUEST','STUDY_DESCRIPTION','REQUESTOR_NAME','LOADED_TO_JIRA','ATTACH'])

daily_req_df=loaded_reqs_df.loc[loaded_reqs_df['LOADED_TO_JIRA'].isnull()]

C:\Users\rybad\AppData\Local\Continuum\anaconda3\lib\site-packages\urllib3\connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\Users\rybad\AppData\Local\Continuum\anaconda3\lib\site-packages\urllib3\connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\Users\rybad\AppData\Local\Continuum\anaconda3\lib\site-packages\urllib3\connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\Users\rybad\AppData\Local\Continuum\a

In [4]:
# #############################################################################
# JIRA api setup variables
# #############################################################################

#USERNAME = os.getenv("USERNAME")
USERNAME='rybad'
JIRA_TOKEN='KB7yNU0HwTq0uARQBfNp75AA'
#JIRA_AUTH = (f"{ USERNAME }@email.chop.edu", os.getenv("JIRA_TOKEN"))
JIRA_AUTH = (f"{ USERNAME }@email.chop.edu", JIRA_TOKEN)

chop_jira = jira.JIRA("https://chopdna.atlassian.net", basic_auth=JIRA_AUTH)

# #############################################################################
# Write New Requests to JIRA
# #############################################################################

cr='\r\n'

for _,row in daily_req_df.iterrows():
    summary = row["REQUEST_TITLE"]
    description = row["STUDY_DESCRIPTION"]    
    requestor = row["REQUESTOR_NAME"]
    attach = drow["ATTACH"]
    new_issue = chop_jira.create_issue(
        project="DA",
        summary=summary,
        description=str(description)+chr(10)+f' Requested By:{requestor}'+chr(10)+f' Attachment:{attach}',
        labels=['Cardiac'],
        issuetype={"name": "Task"},
    )

2022-02-01 16:33:45 - HTTPSConnectionPool(host='chopdna.atlassian.net', port=443): Max retries exceeded with url: /rest/api/2/serverInfo (Caused by SSLError(SSLError("bad handshake: Error([('SSL routines', 'tls_process_server_certificate', 'certificate verify failed')],)",),)) while doing GET https://chopdna.atlassian.net/rest/api/2/serverInfo [{'params': None, 'headers': {'User-Agent': 'python-requests/2.23.0', 'Accept-Encoding': 'gzip, deflate', 'Accept': 'application/json,*.*;q=0.9', 'Connection': 'keep-alive', 'Cache-Control': 'no-cache', 'Content-Type': 'application/json', 'X-Atlassian-Token': 'no-check'}}]
2022-02-01 16:33:45 - Got ConnectionError [HTTPSConnectionPool(host='chopdna.atlassian.net', port=443): Max retries exceeded with url: /rest/api/2/serverInfo (Caused by SSLError(SSLError("bad handshake: Error([('SSL routines', 'tls_process_server_certificate', 'certificate verify failed')],)",),))] errno:None on GET https://chopdna.atlassian.net/rest/api/2/serverInfo
{'response

AttributeError: 'SSLError' object has no attribute 'headers'

In [ ]:
# #############################################################################
# set REDCap records to True for "Loaded to JIRA"
# #############################################################################

daily_req_df['LOADED_TO_JIRA']=1
daily_req_df.index.name = 'cc_data_request_id'
daily_req_df['cc_data_request_id'] = daily_req_df.index
daily_req_df.columns = map(str.lower, daily_req_df.columns)
daily_req_upd = daily_req_df[["cc_data_request_id", "loaded_to_jira"]]
print(daily_req_upd)
response = project.import_records(daily_req_upd)